In [12]:
import os
import glob
import h5py
import numpy as np
from keras import backend as K

import keras
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten, Dense, Activation, Reshape

In [17]:
keras.backend.set_image_dim_ordering('tf')
weights_path = 'yolo-tiny.weights'

In [31]:
model = Sequential()
model.add(Convolution2D(16, 3, 3,input_shape=(448,448,3),border_mode='same',subsample=(1,1)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32,3,3 ,border_mode='same', activation=LeakyReLU(alpha=0.1)))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(64,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(128,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(256,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(512,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(1024,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Convolution2D(1024,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Convolution2D(1024,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Flatten())
model.add(Dense(256))
model.add(Dense(4096))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(1470))

In [37]:
def say(*words, verbalise=False):
    if verbalise:
        print(list(words))

def convert_darkweights2keras(model, weigths_path, verbalise=False):
    data = np.fromfile(weights_path, np.float32)
    data = data[4:]
    say("weights shape : ", data.shape, verbalise=verbalise)
    idx = 0
    for i,layer in enumerate(model.layers):
        shape = [w.shape for w in layer.get_weights()]
        if shape != []:
            kshape,bshape = shape
            bia = data[idx:idx+np.prod(bshape)].reshape(bshape)
            idx += np.prod(bshape)
            ker = data[idx:idx+np.prod(kshape)].reshape(kshape)
            idx += np.prod(kshape)
            layer.set_weights([ker,bia])
    say("convert np weights file -> kears models", verbalise=verbalise)

In [38]:
convert_darkweights2keras(model, weights_path, verbalise=True)

['weights shape : ', (45089374,)]
['convert np weights file -> kears models']


In [28]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_14 (Convolution2D) (None, 448, 448, 16)  448         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
leakyrelu_15 (LeakyReLU)         (None, 448, 448, 16)  0           convolution2d_14[0][0]           
____________________________________________________________________________________________________
maxpooling2d_11 (MaxPooling2D)   (None, 224, 224, 16)  0           leakyrelu_15[0][0]               
____________________________________________________________________________________________________
convolution2d_15 (Convolution2D) (None, 224, 224, 32)  4640        maxpooling2d_11[0][0]            
___________________________________________________________________________________________